# Sudoku miniproject

Below you will find the most elegant solution of sudoku solver coded by Peter Norvig. It's a good notation and implementation of two (simple) techniques that are enough to solve sudoku in a reasonable time.

You can find the original post here: http://norvig.com/sudoku.html

The best website about sudoku: http://www.sudokudragon.com/sudokutheory.htm

### Sudoku Notation and Preliminary Notions

First we have to agree on some notation. A Sudoku puzzle is a grid of 81 squares; the majority of enthusiasts label the columns 1-9, the rows A-I, and call a collection of nine squares (column, row, or box) a unit and the squares that share a unit the peers. A puzzle leaves some squares blank and fills others with digits, and the whole idea is:
A puzzle is solved if the squares in each unit are filled with a permutation of the digits 1 to 9.
That is, no digit can appear twice in a unit, and every digit must appear once. This implies that each square must have a different value from any of its peers. Here are the names of the squares, a typical puzzle, and the solution to the puzzle:

```
 A1 A2 A3| A4 A5 A6| A7 A8 A9    4 . . |. . . |8 . 5     4 1 7 |3 6 9 |8 2 5 
 B1 B2 B3| B4 B5 B6| B7 B8 B9    . 3 . |. . . |. . .     6 3 2 |1 5 8 |9 4 7
 C1 C2 C3| C4 C5 C6| C7 C8 C9    . . . |7 . . |. . .     9 5 8 |7 2 4 |3 1 6 
---------+---------+---------    ------+------+------    ------+------+------
 D1 D2 D3| D4 D5 D6| D7 D8 D9    . 2 . |. . . |. 6 .     8 2 5 |4 3 7 |1 6 9 
 E1 E2 E3| E4 E5 E6| E7 E8 E9    . . . |. 8 . |4 . .     7 9 1 |5 8 6 |4 3 2 
 F1 F2 F3| F4 F5 F6| F7 F8 F9    . . . |. 1 . |. . .     3 4 6 |9 1 2 |7 5 8 
---------+---------+---------    ------+------+------    ------+------+------
 G1 G2 G3| G4 G5 G6| G7 G8 G9    . . . |6 . 3 |. 7 .     2 8 9 |6 4 3 |5 7 1 
 H1 H2 H3| H4 H5 H6| H7 H8 H9    5 . . |2 . . |. . .     5 7 3 |2 9 1 |6 8 4 
 I1 I2 I3| I4 I5 I6| I7 I8 I9    1 . 4 |. . . |. . .     1 6 4 |8 7 5 |2 9 3 
```

Every square has exactly 3 units and 20 peers. For example, here are the units and peers for the square C2:

```
    A2   |         |                    |         |            A1 A2 A3|         |         
    B2   |         |                    |         |            B1 B2 B3|         |         
    C2   |         |            C1 C2 C3| C4 C5 C6| C7 C8 C9   C1 C2 C3|         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    D2   |         |                    |         |                    |         |         
    E2   |         |                    |         |                    |         |         
    F2   |         |                    |         |                    |         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    G2   |         |                    |         |                    |         |         
    H2   |         |                    |         |                    |         |         
    I2   |         |                    |         |                    |         |    
```

In [275]:
#notation

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u]) 
             for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)

print(units['A1'])
#print(peers['A1'])
#print(unitlist)
row_p,col_p,box_p = units['A1']
print(row_p)

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'], ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'], ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]
['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']


### Sudoku grid
Now that we have squares, units, and peers, the next step is to define the Sudoku playing grid. Actually we need two representations: First, a textual format used to specify the initial state of a puzzle; we will reserve the name grid for this. Second, an internal representation of any state of a puzzle, partially solved or complete; this we will call a values collection because it will give all the remaining possible values for each square. For the textual format (grid) we'll allow a string of characters with 1-9 indicating a digit, and a 0 or period specifying an empty square. All other characters are ignored (including spaces, newlines, dashes, and bars). So each of the following three grid strings represent the same puzzle:



In [276]:
# parser
def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))

def display(values):
    "Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '') for c in cols))
        if r in 'CF': print(line)
    print()

# sample sudoku
grid1 = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
display(grid_values(grid2))

4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 



### Parser
Now for values. One might think that a 9 x 9 array would be the obvious data structure. But squares have names like 'A1', not (0,0). Therefore, values will be a dict with squares as keys. The value of each key will be the possible digits for that square: a single digit if it was given as part of the puzzle definition or if we have figured out what it must be, and a collection of several digits if we are still uncertain. This collection of digits could be represented by a Python set or list, but I chose instead to use a string of digits (we'll see why later). So a grid where A1 is 7 and C7 is empty would be represented as {'A1': '7', 'C7': '123456789', ...}.

In [277]:
def parse_grid(grid):
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    values = dict((s, digits) for s in squares)
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## (Fail if we can't assign d to square s.)
    return values

grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
sudoku_values = (grid_values(grid2))
                 
print(sudoku_values)

{'A1': '4', 'A2': '.', 'A3': '.', 'A4': '.', 'A5': '.', 'A6': '.', 'A7': '8', 'A8': '.', 'A9': '5', 'B1': '.', 'B2': '3', 'B3': '.', 'B4': '.', 'B5': '.', 'B6': '.', 'B7': '.', 'B8': '.', 'B9': '.', 'C1': '.', 'C2': '.', 'C3': '.', 'C4': '7', 'C5': '.', 'C6': '.', 'C7': '.', 'C8': '.', 'C9': '.', 'D1': '.', 'D2': '2', 'D3': '.', 'D4': '.', 'D5': '.', 'D6': '.', 'D7': '.', 'D8': '6', 'D9': '.', 'E1': '.', 'E2': '.', 'E3': '.', 'E4': '.', 'E5': '8', 'E6': '.', 'E7': '4', 'E8': '.', 'E9': '.', 'F1': '.', 'F2': '.', 'F3': '.', 'F4': '.', 'F5': '1', 'F6': '.', 'F7': '.', 'F8': '.', 'F9': '.', 'G1': '.', 'G2': '.', 'G3': '.', 'G4': '6', 'G5': '.', 'G6': '3', 'G7': '.', 'G8': '7', 'G9': '.', 'H1': '5', 'H2': '.', 'H3': '.', 'H4': '2', 'H5': '.', 'H6': '.', 'H7': '.', 'H8': '.', 'H9': '.', 'I1': '1', 'I2': '.', 'I3': '4', 'I4': '.', 'I5': '.', 'I6': '.', 'I7': '.', 'I8': '.', 'I9': '.'}


# Constraint Propagation - original code
The function parse_grid calls assign(values, s, d). We could implement this as values[s] = d, but we can do more than just that. Those with experience solving Sudoku puzzles know that there are two important strategies that we can use to make progress towards filling in all the squares:

(1) If a square has only one possible value, then eliminate that value from the square's peers. 

(2) If a unit has only one possible place for a value, then put the value there.

As an example of strategy (1) if we assign 7 to A1, yielding {'A1': '7', 'A2':'123456789', ...}, we see that A1 has only one value, and thus the 7 can be removed from its peer A2 (and all other peers), giving us {'A1': '7', 'A2': '12345689', ...}. As an example of strategy (2), if it turns out that none of A3 through A9 has a 3 as a possible value, then the 3 must belong in A2, and we can update to {'A1': '7', 'A2':'3', ...}. These updates to A2 may in turn cause further updates to its peers, and the peers of those peers, and so on. This process is called constraint propagation.

The function assign(values, s, d) will return the updated values (including the updates from constraint propagation), but if there is a contradiction--if the assignment cannot be made consistently--then assign returns False. For example, if a grid starts with the digits '77...' then when we try to assign the 7 to A2, assign would notice that 7 is not a possibility for A2, because it was eliminated by the peer, A1.

It turns out that the fundamental operation is not assigning a value, but rather eliminating one of the possible values for a square, which we implement with eliminate(values, s, d). Once we have eliminate, then assign(values, s, d) can be defined as "eliminate all the values from s except d".

In [278]:
def assign(values, s, d):
    """Eliminate all the other values (except d) from values[s] and propagate.
    Return values, except return False if a contradiction is detected."""
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False

def eliminate(values, s, d):
    """Eliminate d from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected."""
    if d not in values[s]:
        return values ## Already eliminated
    values[s] = values[s].replace(d,'')
    ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
    if len(values[s]) == 0:
        return False ## Contradiction: removed last value
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    ## (2) If a unit u is reduced to only one place for a value d, then put it there.
    for u in units[s]:
        dplaces = [s for s in u if d in values[s]]
        if len(dplaces) == 0:
            return False ## Contradiction: no place for this value
        elif len(dplaces) == 1:
            # d can only be in one place in unit; assign it there
            if not assign(values, dplaces[0], d):
                return False
    return values

def parse_grid(grid):
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    values = dict((s, digits) for s in squares)
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## (Fail if we can't assign d to square s.)
    return values

### Test run

In [279]:
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
sudoku_values = (parse_grid(grid1))
display(sudoku_values)

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 



# Constraint Propagation - code refactoring

Try to decompose* eliminate function into smaller one.
* Decomposition is a process by which you can break down one complex function into multiple smaller functions. By doing this, you can solve for functions in shorter, easier-to-understand pieces.

# Algorithms
http://www.sudokudragon.com/tutorialnakedtwins.htm

In [280]:
# todo
def parse_grid_2(grid):
    #uzupełnić słownik wartościami jeśli należą do zbioru digit a jeśli nie (czyli kropa lub 0)
    #to wypełnić wszystkimi możliwościami
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    values = dict((s, digits) for s in squares)
    for s,d in grid_values(grid).items():
        if d in digits:
            values[s] = d
    return values


def eliminate(values):
    """
    Iterate through all squares and every time 
       if there is a square with one value, 
       then eliminate this value from the peers

    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    for s in squares:
        if len(values[s]) == 0:
            return False ## Contradiction: removed last value
        elif len(values[s]) == 1:
            d2 = values[s]
            for s2 in peers[s]:
                values[s2] = values[s2].replace(d2,'')
    return values    
    
def only_choice(values):
    """
    Iterate through all squares and every time
        if there is a square with a value that only fits in one square, 
        assign the value to this square

    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    for u in unitlist:
        for d in digits:
            dplaces = [s for s in u if d in values[s]]
            if len(dplaces) == 0:
                return False ## Contradiction: no place for this value
            elif len(dplaces) == 1:
                # d can only be in one place in unit; assign it there
                # tu można potem dodać, żeby nie brał pod uwagę już rozw. squares
                values[dplaces[0]] = d    
    return values

def naked_twins(values):
    """
    eliminate values using the naked twins strategy
    
    input: A sudoku in dictionary form.
    output: The resulting sudoku in dictionary form.
    """
    # Squares with two possible value are potential twins
    potential_twins = [s for s in values.keys() if len(values[s]) == 2]
    # Collect boxes that have the same elements
    naked_twins = [[s1,s2] for s1 in potential_twins 
                    for s2 in peers[s1] 
                    if set(values[s1])==set(values[s2]) ]

    # For each pair of naked twins,
    for i in range(len(naked_twins)):
        s1 = naked_twins[i][0]
        s2 = naked_twins[i][1]
        # 1- compute intersection of peers
        peers1 = set(peers[s1])
        peers2 = set(peers[s2])
        common_peers = peers1 & peers2
        # 2- Delete the two digits in naked twins from all common peers.
        for peer_val in common_peers:
            if len(values[peer_val])>2:
                for rm_val in values[s1]:
                    values[peer_val].replace(rm_val,'')
    return values

def sub_group(values):
    """Row and column subgroup elimination using Sub-group exclusion."""
    #index==0 for row subgroups, and index==1 for column subgroups
    for index,RC in enumerate(zip(['ABCDEFGHI', '123456789'], [['123','456', '789'], ['ABC', 'DEF', 'GHI']])):
        R,C = RC  
        for r in R: #index==0 start with "A" | index==1 start with "1"
            for c in C: #index==0 start with "123" | index==1 start with "ABC"
                if index==0:
                    _3p = cross(r, c) # A1 A2 A3
                    first_pos = _3p[0] # A1
                else:
                    _3p = cross(c, r) # A1 B1 C1
                    first_pos = _3p[0] # A1
                col_p,row_p,box_p = units[first_pos]
                #index==0 row_p A1 A2 A3 A4 A5 A6 A7 A8 A9
                #index==0 col_p A1 B1 C1 D1 E1 F1 G1 H1 I1
                #index==0 sq_p A1 B1 C1 A2 B2 C2 A3 B3 C3
                if index==0:
                    _6p = (set(row_p)-set(_3p)) # A4 A5 A6 A7 A8 A9
                else:
                    _6p = (set(col_p)-set(_3p)) # D1 E1 F1 G1 H1 I1
                for s in _3p: #index==0 start with A1
                    v = values[s] # possible values for A1
                    if len(v)>1:
                        for vv in v: # for all possible values for A1
                            if vv not in [i for j in _6p for i in values[j]]: # not in values for A4 A5 A6 A7 A8 A9
                                for p in box_p: # for all possible values in square with A1
                                    if (not p in _3p) and len(values[p])>1 and vv in values[p]:
                                        values[p] = values[p].replace(vv, '') # delete vv from p if present
    return values

def reduce_puzzle(values):
    """
    Solve sudoku using eliminate() and only_choice()
    
    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    solved = False
    while not solved:
        solved_values_previous = len([s for s in values.keys() if len(values[s]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        values = sub_group(values)
        solved_values_after = len([s for s in values.keys() if len(values[s]) == 1])
        solved = solved_values_after == solved_values_previous
        if len([s for s in values.keys() if len(values[s]) == 0]):
            return False
    return values

def search(values):
    "Using depth-first search and propagation - search tree."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values == False:
        return values
    
    if all(len(values[s]) == 1 for s in squares):
        return values ## Solved!

    n,s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

# Sudoku generator

In [281]:
import random
import time

def fill_random_square(squares, grid):
    #find random square and fill it with random number
    random_square = random.choice(squares)
    random_number = str(random.randint(1, 9))
    grid[random_square] = random_number
    return grid

def remove_random_squares(squares, grid, removed_number):
    #remove number from "removed_number" of squares
    list=[]
    for i in range(removed_number):
        r = random.choice(squares)
        if r not in list: list.append(r)
        grid[r] = '.' 
    return grid

def reparse_grid(grid):
    #change dict form of grid to string
    sudoku = []
    for k, v in grid.items():
        if v.isnumeric() or v == ".":
            sudoku.append(v)
    return ''.join(sudoku)

def generate_sudoku(squares):
    empty_grid = "................................................................................."
    init_empty_grid = parse_grid_2(empty_grid)
    init_filled_empty_grid = fill_random_square(squares, init_empty_grid)
    if search(init_filled_empty_grid):
        solved_grid = search(init_filled_empty_grid)
        #display(solved_grid)
        generated_grid = remove_random_squares(squares, solved_grid, 64)
        #display(generated_grid)
        sudoku = reparse_grid(generated_grid)
        return sudoku
    else:
        return False

def find_hard_sudoku(sudoku_dict, sudoku_number):
    #find "sudoku_number" of the hardest sudoku's (with the highest execution time)
    hard_sudoku_dict = {}
    for i in range(sudoku_number):
        max_time = max(sudoku_dict.keys())
        #print(max_time)
        hard_sudoku_dict[max_time] = sudoku_dict[max_time]
        del sudoku_dict[max_time]
    return hard_sudoku_dict
    
def clear_file():
    f = open("sudoku.txt", "w")
    if f:
        f.write("")
    f.close() 

def write_sudoku_to_file(sudoku):
    f = open("sudoku.txt", "a")
    if f:
        f.write(sudoku + "\n")
    f.close()

def read_sudoku_from_file():
    #read all sudoku's from file and add them to list
    sudoku_list = []
    f = open("sudoku.txt", "r")
    if f:
        for i in f:
            sudoku_list.append(i)
    f.close()
    return sudoku_list
    
sudoku_dict = {}
#looking for sudoku's that can be solved by above algorithm
print("Execution step:")
for i in range(100):
    try:
        sudoku = generate_sudoku(squares)    
    except:
        continue
    else:
        parsed_sudoku = parse_grid_2(sudoku)
        start = time.time()
        try:
            solved_sudoku = search(parsed_sudoku)
        except:
            continue
        else:
            end = time.time()
            print(i)
            elapsed_time = round(end-start, 5)
            sudoku_dict[elapsed_time] = sudoku
            
sudoku_number = 10
hard_sudoku_dict = find_hard_sudoku(sudoku_dict, sudoku_number)
clear_file()
for sudoku in hard_sudoku_dict.values():
    write_sudoku_to_file(sudoku)

Execution step:
0
1
3
4
5
6
8
9
10
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
73
74
75
76
77
78
79
80
81
82
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


# Sudoku solution

In [283]:
import time

grid1 = "003020600900305001001806400008102900700000008006708200002609500800203009005010300"
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
grid3 = "2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3"
grid4 = "...............9..97.3......1..6.5....47.8..2.....2..6.31..4......8..167.87......"

#read sudoku's from file and solve them
sudoku_list = read_sudoku_from_file()
for i in sudoku_list:
    values_init = parse_grid_2(i)
    start = time.time()
    values = search(values_init)
    print("Sudoku grid:")
    display(values)
    end = time.time()
    print("Execution time:")
    print(end-start)
    print("\n")

Sudoku grid:
1 5 7 |2 4 6 |8 3 9 
6 3 2 |5 8 9 |7 1 4 
8 9 4 |1 3 7 |2 5 6 
------+------+------
4 1 3 |7 5 2 |9 6 8 
7 2 5 |6 9 8 |3 4 1 
9 8 6 |4 1 3 |5 7 2 
------+------+------
3 7 1 |8 2 4 |6 9 5 
2 4 9 |3 6 5 |1 8 7 
5 6 8 |9 7 1 |4 2 3 

Execution time:
0.0742032527923584


Sudoku grid:
4 5 6 |1 2 3 |7 8 9 
7 8 9 |4 5 6 |1 2 3 
1 2 3 |7 8 9 |4 5 6 
------+------+------
3 1 2 |6 7 4 |8 9 5 
8 7 5 |9 1 2 |3 6 4 
6 9 4 |5 3 8 |2 1 7 
------+------+------
2 3 7 |8 6 5 |9 4 1 
5 4 1 |2 9 7 |6 3 8 
9 6 8 |3 4 1 |5 7 2 

Execution time:
0.052720069885253906


Sudoku grid:
1 2 4 |5 3 6 |7 8 9 
3 5 6 |7 8 9 |1 2 4 
7 8 9 |1 2 4 |3 5 6 
------+------+------
2 4 1 |6 7 3 |8 9 5 
5 9 7 |4 1 8 |2 6 3 
8 6 3 |2 9 5 |4 1 7 
------+------+------
4 3 2 |9 5 1 |6 7 8 
6 1 5 |8 4 7 |9 3 2 
9 7 8 |3 6 2 |5 4 1 

Execution time:
0.04556703567504883


Sudoku grid:
3 4 6 |7 8 9 |2 1 5 
5 8 7 |1 2 4 |3 6 9 
1 2 9 |3 5 6 |4 8 7 
------+------+------
7 6 3 |9 4 8 |5 2 1 
8 9 2 |5 1 7 |6 3 4 
4 1 5 |2 6 3